# so6 air traffic file to CesiumJS CZML

In [1]:
import pandas as pd
import random
import json

#### <br> Build CZML from pandas dataframe

In [2]:
def get_start_stop(df):
    starttime = str(min(df['date_begin_time_begin'])).replace(" ", "T").replace(".000", "Z")
    stoptime = str(max(df['date_end_time_end'])).replace(" ", "T").replace(".000", "Z")
    availability = starttime + "/" + stoptime
    return (starttime, stoptime, availability)

def get_4D_coords(group):
    coords = []
    t = 0
    for _,row in group.iterrows():
        coords.extend([t, row['lon_begin'], row['lat_begin'], row['fl_begin']])
        t += row['duration']
    return coords

class CZMLBuilder:
    
    def __init__(self, df, global_id, global_name, global_version, global_author, 
                 default_time_multiplier=100, path_outline_width=3, path_width=3, 
                 path_lead_time=0, path_trail_time=100000, path_resolution=5,
                 point_outline_width=2, point_pixel_size=6, 
                 point_height_reference="NONE"):
        
        self.path_outline_width = path_outline_width
        self.path_width = path_width
        self.path_lead_time = path_lead_time
        self.path_trail_time = path_trail_time
        self.path_resolution = path_resolution
        self.point_outline_width = point_outline_width
        self.point_pixel_size = point_pixel_size
        self.point_height_reference = point_height_reference
        starttime, _, availability = get_start_stop(df)
        self.df = df
        self.groups = self.df.groupby('flight_identifier')
        self.czml_output = []
        
        global_element = {
                        "id": global_id, 
                        "name": global_name, 
                        "version": global_version, 
                        "author": global_author, "clock": {"interval": availability, 
                                                            "currentTime": starttime, 
                                                            "multiplier": default_time_multiplier
                                                          }
                        }
        self.czml_output.append(global_element)
    
    def _get_path(self, group, path_id, starttime, availability, color):
        path = {
            "id": path_id,
            "availability": availability,
            "position": {
                "epoch": starttime,
                "cartographicDegrees": get_4D_coords(group)
            },
            "path": {
                "material": {
                    "polylineOutline": {
                        "color": {
                            "rgba": color
                        },
                        "outlineColor": {
                            "rgba": color 
                        },
                        "outlineWidth": self.path_outline_width
                    }
                },
                "width": self.path_width,
                "leadTime": self.path_lead_time,
                "trailTime": self.path_trail_time,
                "resolution": self.path_resolution
            }
        }
        return path
    
    def _get_point(self, group, point_id, starttime, availability, color):
        point = {
            "id": point_id,
            "availability": availability,

            "position": {
                "epoch": starttime,
                "cartographicDegrees": get_4D_coords(group)
            },
            "point": {
                "color": {
                    "rgba": [255,255,255,200] # white center instead of color
                },
                "outlineColor": {
                    "rgba": color
                },
                "outlineWidth": self.point_outline_width,
                "pixelSize": self.point_pixel_size,
                "heightReference": self.point_height_reference
            }   
        }
        return point
    
    def save(self, file_path, path=True, point=True):
        for name, group in self.groups:
            color = [random.randint(0,255), random.randint(0,255), random.randint(0,255), 150]
            starttime, _, availability = get_start_stop(group)
            if path:
                self.czml_output.append(self._get_path(group, name, starttime, availability, color))
            if point:
                self.czml_output.append(self._get_point(group, name, starttime, availability, color))
        with open(file_path, 'w') as outfile:
            json.dump(self.czml_output, outfile)
        

#### <br> Load so6 file as a pandas dataframe

In [4]:
YOUR_SO6_FILE_PATH = 'your_so6_file.so6'
columns = ['segment_identifier', 'flight_origin', 'flight_destination', 'aircraft_type', 
           'time_begin', 'time_end', 'fl_begin', 'fl_end', 'status', 'callsign', 
           'date_begin', 'date_end', 'lat_begin', 'lon_begin', 'lat_end', 'lon_end',
           'flight_identifier', 'sequence', 'length', 'parity']
parser = lambda x,y: pd.datetime.strptime(x + y, '%y%m%d%H%M%S')

df = pd.read_csv(YOUR_SO6_FILE_PATH, sep=' ', header=None, names=columns, 
                 parse_dates=[['date_begin','time_begin'],['date_end','time_end']], date_parser=parser)

df.query('length > 0', inplace=True) # filter null segments
df['duration'] = (df['date_end_time_end'] - df['date_begin_time_begin']).dt.total_seconds()
df['fl_begin'] = df['fl_begin'].apply(lambda x: x * 30.48) # FL to meters
lat_lon_cols = ['lat_begin','lon_begin','lat_end','lon_end']
df[lat_lon_cols] = df[lat_lon_cols].apply(lambda x: x / 60)
df.drop(columns=['segment_identifier'], inplace=True)
df.head()

,date_begin_time_begin,date_end_time_end,flight_origin,flight_destination,aircraft_type,fl_begin,fl_end,status,callsign,lat_begin,lon_begin,lat_end,lon_end,flight_identifier,sequence,length,parity,duration
0,2018-03-01 14:25:00,2018-03-01 14:25:44,ENXU,ENXN,S92,60.96,1,2,HKS147,59.445833,2.344444,59.462500,2.365278,215700734,1,1.184730,0,44.0
3,2018-03-01 06:41:00,2018-03-01 06:41:45,LEAL,EGPD,B738,30.48,13,0,RYR8005,38.282222,-0.558056,38.275278,-0.525556,215689750,1,1.586456,0,45.0
4,2018-03-01 06:41:45,2018-03-01 06:41:55,LEAL,EGPD,B738,396.24,20,0,RYR8005,38.275278,-0.525556,38.273056,-0.514722,215689750,2,0.527418,0,10.0
5,2018-03-01 06:41:55,2018-03-01 06:42:03,LEAL,EGPD,B738,609.60,20,2,RYR8005,38.273056,-0.514722,38.270556,-0.503889,215689750,3,0.531892,0,8.0
6,2018-03-01 06:42:03,2018-03-01 06:42:18,LEAL,EGPD,B738,609.60,25,0,RYR8005,38.270556,-0.503889,38.266111,-0.481944,215689750,4,1.067580,0,15.0


#### <br> Filter dataframe (if needed)

In [5]:
df.query('flight_origin == "LFBO"', inplace=True) # ex: only flights from LFBO
df.head()

,date_begin_time_begin,date_end_time_end,flight_origin,flight_destination,aircraft_type,fl_begin,fl_end,status,callsign,lat_begin,lon_begin,lat_end,lon_end,flight_identifier,sequence,length,parity,duration
4501,2018-03-01 10:21:00,2018-03-01 10:21:20,LFBO,LFPO,A321,152.4,10,0,AFR81VM,43.635000,1.367778,43.626944,1.373889,215694215,1,0.551402,0,20.0
4502,2018-03-01 10:21:20,2018-03-01 10:22:01,LFBO,LFPO,A321,304.8,20,0,AFR81VM,43.626944,1.373889,43.618889,1.380000,215694215,2,0.551420,0,41.0
4503,2018-03-01 10:22:01,2018-03-01 10:22:39,LFBO,LFPO,A321,609.6,30,0,AFR81VM,43.618889,1.380000,43.603056,1.392222,215694215,3,1.088311,0,38.0
4504,2018-03-01 10:22:39,2018-03-01 10:22:58,LFBO,LFPO,A321,914.4,35,0,AFR81VM,43.603056,1.392222,43.594167,1.390556,215694215,4,0.538227,0,19.0
4505,2018-03-01 10:22:58,2018-03-01 10:23:45,LFBO,LFPO,A321,1066.8,50,0,AFR81VM,43.594167,1.390556,43.576389,1.387222,215694215,5,1.076460,0,47.0


#### <br> Write the CZML file from the so6 dataframe

In [13]:
czml_builder = CZMLBuilder(df, "document", "test_so6", "1.0", "your_name")
czml_builder.save('test.czml')